In [1]:

import torch

from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model.to(device)
model.eval()
model.zero_grad()
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id

"""++++++++++++++++++这几个函数是计算f1 score 数值的，代码是抄的，千万不能改！+++++++++++++++++"""
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    ids = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)
                    ids.append(qa['id'])

    return contexts, questions, answers, ids

train_contexts, train_questions, train_answers, train_ids = read_squad(r'D:\software\github\GZK_Code\XAI\2022.03.03\squad\train-v2.0.json')
val_contexts, val_questions, val_answers, val_ids = read_squad(r'D:\software\github\GZK_Code\XAI\2022.03.03\squad\dev-v2.0.json')

def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)


def get_gold_answers(example):
    """helper function that retrieves all possible true answers from a squad2.0 example"""

    gold_answers = [answer["text"] for answer in example.answers if answer["text"]]

    # if gold_answers doesn't exist it's because this is a negative example -
    # the only correct answer is an empty string
    if not gold_answers:
        gold_answers = [""]

    return gold_answers


"""+++++++++++++++++++++++++++++++++++"""

def predict(inputs):
    output = model(inputs)
    return output.start_logits, output.end_logits


def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]

    # construct reference token ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + \
                    [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

def predict_qt(question, text):
    input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)

    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)

    ground_truth = '13'


    start_scores, end_scores = predict(input_ids)


    return (' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])),float(torch.max(torch.softmax(start_scores[0], dim=0))),float(torch.max(torch.softmax(end_scores[0], dim=0)))
def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""
    import string, re

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))

def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()

    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)

    common_tokens = set(pred_tokens) & set(truth_tokens)

    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0

    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)

    return 2 * (prec * rec) / (prec + rec)



In [2]:
total = sum([param.nelement() for param in model.parameters()])

print("Number of parameter: %.2fM" % (total/1e6))

Number of parameter: 334.09M


In [5]:
acc = 0
evl_dick = {}
failu_num = 0
for i in range(len(val_ids)):
    question = val_questions[i]
    text = val_contexts[i]
    try:
        answer, start_score, end_score = predict_qt(question, text)
        evl_dick[str(val_ids[i])] = answer

    except:
        failu_num+=1
    if i % 500 == 0:
        print(i)

print(acc/1500, "失败", failu_num)


0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500


Token indices sequence length is longer than the specified maximum sequence length for this model (575 > 512). Running this sequence through the model will result in indexing errors


6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
0.0 失败 144


In [7]:
len(evl_dick)

5882

In [8]:
import datetime
time = str(datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S'))
print (time)
json_dick = json.dumps(evl_dick)
filname = time + "answers.txt"
fo = open(filname, "w",encoding='utf-8')
fo.write(json_dick)
fo.close()

2022-03-15-22-41-58


In [4]:
model = BertForQuestionAnswering.from_pretrained('mrm8488/bert-small-finetuned-squadv2')
model.to(device)
model.eval()
model.zero_grad()
tokenizer = BertTokenizer.from_pretrained('mrm8488/bert-small-finetuned-squadv2')


In [5]:
# val_contexts, val_questions, val_answers
acc = 0
for i in range(1500):
    question = train_questions[i]
    text = train_contexts[i]
    answer, start_score, end_score = predict_qt(question, text)
    f1 = compute_f1(answer, train_answers[i]["text"])
    acc += f1
    if i % 500 == 0:
        print(i)

print(acc/1500)

0
500
1000
0.3408622777614493


In [6]:
total = sum([param.nelement() for param in model.parameters()])

print("Number of parameter: %.2fM" % (total/1e6))

Number of parameter: 28.50M


In [7]:
from transformers import DistilBertTokenizerFast
import torch
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = torch.load("sep_train_distilbert.pth")

import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id
# val_contexts, val_questions, val_answers
acc = 0
for i in range(1500):
    question = train_questions[i]
    text = train_contexts[i]
    answer, start_score, end_score = predict_qt(question, text)
    f1 = compute_f1(answer, train_answers[i]["text"])
    acc += f1
    if i % 500 == 0:
        print(i)

print(acc/1500)
total = sum([param.nelement() for param in model.parameters()])

print("Number of parameter: %.2fM" % (total/1e6))

0
500
1000
0.23165809118725164
Number of parameter: 66.36M


In [8]:
model = torch.load("epoch10sepdata_big.pth")

import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id
# val_contexts, val_questions, val_answers
acc = 0
for i in range(1500):
    question = train_questions[i]
    text = train_contexts[i]
    answer, start_score, end_score = predict_qt(question, text)
    f1 = compute_f1(answer, train_answers[i]["text"])
    acc += f1
    if i % 500 == 0:
        print(i)

print(acc/1500)
total = sum([param.nelement() for param in model.parameters()])

print("Number of parameter: %.2fM" % (total/1e6))

print("分着测")
import nltk as tk
f1s = []
f1 = []
start_scores = []
end_scores = []
t_start_score = []
t_end_score = []
for i in range(1500):
    text = train_contexts[i]
    tokens = tk.sent_tokenize(text)
    for token in tokens:
        question = train_questions[i]
        answer, start_score, end_score = predict_qt(question, token)
        t_start_score.append(start_score)
        t_end_score.append(end_score)
        f1.append(compute_f1(answer, train_answers[i]["text"]))
    f1s.append(f1)
    start_scores.append(t_start_score)
    end_scores.append(t_end_score)
    f1 = []
    t_start_score = []
    t_end_score = []

    if i % 500 == 0:
        print(i)
        
temp = 0
print(len(f1s))
for i in f1s:
    temp+=max(i)
print(temp/1500)

0
500
1000
0.4189422020871912
Number of parameter: 66.36M
分着测
0
500
1000
1500
0.6277082048561468


In [9]:
a = [1,2,3,4,5,6]
a.index(max(a))
temp  = 0
start_num = []
end_num = []
for i in range(len(f1s)):
    if f1s[i].index(max(f1s[i])) == start_scores[i].index(max(start_scores[i])):
        start_num.append(i)
    if f1s[i].index(max(f1s[i])) == end_scores[i].index(max(end_scores[i])):
        end_num.append(i)
print(len(start_num)/len(f1s), len(end_num)/len(f1s))


0.4686666666666667 0.4726666666666667


## 分着预测看看

In [10]:
import nltk as tk
f1s = []
f1 = []
for i in range(1500):
    text = train_contexts[i]
    tokens = tk.sent_tokenize(text)
    for token in tokens:
        question = train_questions[i]
        answer, start_score, end_score = predict_qt(question, token)
        f1.append(compute_f1(answer, train_answers[i]["text"]))
    f1s.append(f1)
    f1 = []

    if i % 500 == 0:
        print(i)



0
500
1000


In [11]:
temp = 0
print(len(f1s))
for i in f1s:
    temp+=max(i)
print(temp/1500)

1500
0.6277082048561468
